<a href="https://colab.research.google.com/github/hila-chefer/Transformer-Explainability/blob/main/BERT_explainability.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !git clone https://github.com/hila-chefer/Transformer-Explainability.git

import os
# os.chdir(f'./Transformer-Explainability')

# !pip install -r requirements.txt
# !pip install captum

In [ ]:
# !pip install captum==0.6.0
# !pip install matplotlib==3.3.2

In [9]:
from transformers import BertTokenizer
from BERT_explainability.modules.BERT.ExplanationGenerator import Generator
from BERT_explainability.modules.BERT.BertForSequenceClassification import BertForSequenceClassification
from transformers import BertTokenizer
from BERT_explainability.modules.BERT.ExplanationGenerator import Generator
from transformers import AutoTokenizer

from captum.attr import visualization
import torch
import numpy as np
device = "cuda:1" if torch.cuda.is_available() else "cpu"
print(f'running on {device}')

running on cuda:1


In [10]:
model = BertForSequenceClassification.from_pretrained("textattack/bert-base-uncased-SST-2").to(device)
model.eval()
tokenizer = AutoTokenizer.from_pretrained("textattack/bert-base-uncased-SST-2")
# initialize the explanations generator
explanations = Generator(model)

classifications = ["NEGATIVE", "POSITIVE"]


In [ ]:
model

## interpret from 2nd paper
writing the interpretation that is implemented in the followup paper with intent to compare between the two

In [ ]:
attn_blocks=list(dict(model.bert.encoder.layer.named_children()).values())

In [ ]:
attn_blocks[0]

In [5]:
attn_blocks[0].attention.self.attn.shape

NameError: name 'attn_blocks' is not defined

In [ ]:
len(attn_blocks)

In [44]:
start_layer = 0

def interpret(input_ids, model, attention_mask, start_layer_text=start_layer):
    device = model.device
    batch_size = input_ids.shape[0]
    logits = model(input_ids=input_ids, attention_mask=attention_mask)[0]
    probs = logits.softmax(dim=-1).detach().cpu().numpy()

    # index = [i for i in range(batch_size)]
    index = np.argmax(probs, axis=-1)
    one_hot = np.zeros((logits.shape[0], logits.shape[1]), dtype=np.float32)
    one_hot[torch.arange(logits.shape[0]), index] = 1
    one_hot = torch.from_numpy(one_hot).requires_grad_(True)
    one_hot = torch.sum(one_hot.cuda(device) * logits)
    model.zero_grad()

    # text_attn_blocks = list(dict(model.transformer.resblocks.named_children()).values())
    text_attn_blocks = list(dict(model.bert.encoder.layer.named_children()).values())

    if start_layer_text == -1: 
      # calculate index of last layer 
      start_layer_text = len(text_attn_blocks) - 1

    # num_tokens = text_attn_blocks[0].attn_probs.shape[-1]
    num_tokens = text_attn_blocks[0].attention.self.attn.shape[-1]
    R_text = torch.eye(num_tokens, num_tokens, dtype=text_attn_blocks[0].attention.self.attn.dtype).to(device)
    R_text = R_text.unsqueeze(0).expand(batch_size, num_tokens, num_tokens)
    for i, blk in enumerate(text_attn_blocks):
        if i < start_layer_text:
          continue
        # grad = torch.autograd.grad(one_hot, [blk.attn_probs], retain_graph=True)[0].detach()
        # cam = blk.attn_probs.detach()
        grad = torch.autograd.grad(one_hot, [blk.attention.self.attn], retain_graph=True)[0].detach()
        cam = blk.attention.self.attn.detach()
        cam = cam.reshape(-1, cam.shape[-1], cam.shape[-1])
        grad = grad.reshape(-1, grad.shape[-1], grad.shape[-1])
        cam = grad * cam
        cam = cam.reshape(batch_size, -1, cam.shape[-1], cam.shape[-1])
        cam = cam.clamp(min=0).mean(dim=1)
        R_text = R_text + torch.bmm(cam, R_text)
    expl = R_text
    expl[:, 0, 0] = expl[:, 0].min()
    # expl = (expl - expl.min()) / (expl.max() - expl.min())
    # expl = expl / expl.sum()
    
    
   
    return probs,expl[:,0]

In [49]:
# encode a sentence
text_batch = ["This movie was the best movie I have ever seen! some scenes were ridiculous, but acting was great."]
encoding = tokenizer(text_batch, return_tensors='pt')
input_ids = encoding['input_ids'].to(device)
attention_mask = encoding['attention_mask'].to(device)

# true class is positive - 1
true_class = 1

probs, expl = interpret(input_ids,model,attention_mask,start_layer_text=0)
expl = expl[0]
expl = (expl - expl.min()) / (expl.max() - expl.min())
# expl = expl / expl.sum()
classification = probs.argmax(axis=-1).item()
# get class name
class_name = classifications[classification]
# if the classification is negative, higher explanation scores are more negative
# flip for visualization
if class_name == "NEGATIVE":
  expl *= (-1)

tokens = tokenizer.convert_ids_to_tokens(input_ids.flatten())
print([(tokens[i], expl[i].item()) for i in range(len(tokens))])
vis_data_records = [visualization.VisualizationDataRecord(
                                expl,
                                probs[0][classification],
                                classification,
                                true_class,
                                true_class,
                                1,       
                                tokens,
                                1)]
visualization.visualize_text(vis_data_records)

[('[CLS]', 0.0), ('this', 0.4532674252986908), ('movie', 0.4462960958480835), ('was', 0.21855813264846802), ('the', 0.2666281461715698), ('best', 0.5442412495613098), ('movie', 0.3071017563343048), ('i', 0.14080315828323364), ('have', 0.07363813370466232), ('ever', 0.09055276215076447), ('seen', 0.10591993480920792), ('!', 0.7834613919258118), ('some', 0.011827755719423294), ('scenes', 0.1388983577489853), ('were', 0.0), ('ridiculous', 0.10907001793384552), (',', 0.10549715906381607), ('but', 0.7055017948150635), ('acting', 0.8157413601875305), ('was', 0.4857148826122284), ('great', 1.0), ('.', 0.3004361093044281), ('[SEP]', 0.654569149017334)]


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
1,1 (1.00),1,1.00,"[CLS] this movie was the best movie i have ever seen ! some scenes were ridiculous , but acting was great . [SEP]"


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
1,1 (1.00),1,1.00,"[CLS] this movie was the best movie i have ever seen ! some scenes were ridiculous , but acting was great . [SEP]"


In [46]:
expl

tensor([0.0005, 0.0023, 0.0023, 0.0014, 0.0016, 0.0026, 0.0017, 0.0011, 0.0008,
        0.0009, 0.0009, 0.0036, 0.0006, 0.0011, 0.0005, 0.0010, 0.0009, 0.0033,
        0.0037, 0.0024, 0.0044, 0.0017, 0.0031], device='cuda:1')

## original paper 
### Positive sentiment example

In [4]:
expl.shape

torch.Size([23])

In [7]:
input_ids.shape

torch.Size([1, 23])

In [27]:
# encode a sentence
text_batch = ["This movie was the best movie I have ever seen! some scenes were ridiculous, but acting was great."]
encoding = tokenizer(text_batch, return_tensors='pt')
input_ids = encoding['input_ids'].to(device)
attention_mask = encoding['attention_mask'].to(device)

# true class is positive - 1
true_class = 1

# generate an explanation for the input
expl = explanations.generate_LRP(input_ids=input_ids, attention_mask=attention_mask, start_layer=0)[0]
# normalize scores
expl = (expl - expl.min()) / (expl.max() - expl.min())

# get the model classification
output = torch.nn.functional.softmax(model(input_ids=input_ids, attention_mask=attention_mask)[0], dim=-1)
classification = output.argmax(dim=-1).item()
# get class name
class_name = classifications[classification]
# if the classification is negative, higher explanation scores are more negative
# flip for visualization
if class_name == "NEGATIVE":
  expl *= (-1)

tokens = tokenizer.convert_ids_to_tokens(input_ids.flatten())
print([(tokens[i], expl[i].item()) for i in range(len(tokens))])
vis_data_records = [visualization.VisualizationDataRecord(
                                expl,
                                output[0][classification],
                                classification,
                                true_class,
                                true_class,
                                1,       
                                tokens,
                                1)]
visualization.visualize_text(vis_data_records)

/home/gkoren2/work/git/clones/transformers/src/transformers/modeling_utils.py:866: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
/home/gkoren2/work/git/clones/transformers/src/transformers/modeling_utils.py:866: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


[('[CLS]', 0.0), ('this', 0.4272907078266144), ('movie', 0.30951225757598877), ('was', 0.268684446811676), ('the', 0.339260071516037), ('best', 0.6288697123527527), ('movie', 0.28607645630836487), ('i', 0.1867041438817978), ('have', 0.10146278887987137), ('ever', 0.14219637215137482), ('seen', 0.19010096788406372), ('!', 0.5949175953865051), ('some', 0.0040426114574074745), ('scenes', 0.0334150567650795), ('were', 0.01868431642651558), ('ridiculous', 0.018948297947645187), (',', 0.0), ('but', 0.42987316846847534), ('acting', 0.43847790360450745), ('was', 0.5003233551979065), ('great', 1.0), ('.', 0.014387628063559532), ('[SEP]', 0.08716049790382385)]


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
1,1 (1.00),1,1.00,"[CLS] this movie was the best movie i have ever seen ! some scenes were ridiculous , but acting was great . [SEP]"


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
1,1 (1.00),1,1.00,"[CLS] this movie was the best movie i have ever seen ! some scenes were ridiculous , but acting was great . [SEP]"


#Negative sentiment example

In [ ]:
# encode a sentence
text_batch = ["I really didn't like this movie. Some of the actors were good, but overall the movie was boring."]
encoding = tokenizer(text_batch, return_tensors='pt')
input_ids = encoding['input_ids'].to(device)
attention_mask = encoding['attention_mask'].to(device)

# generate an explanation for the input
expl = explanations.generate_LRP(input_ids=input_ids, attention_mask=attention_mask, start_layer=0)[0]
# normalize scores
expl = (expl - expl.min()) / (expl.max() - expl.min())

# get the model classification
output = torch.nn.functional.softmax(model(input_ids=input_ids, attention_mask=attention_mask)[0], dim=-1)
classification = output.argmax(dim=-1).item()
# get class name
class_name = classifications[classification]
# if the classification is negative, higher explanation scores are more negative
# flip for visualization
if class_name == "NEGATIVE":
  expl *= (-1)

tokens = tokenizer.convert_ids_to_tokens(input_ids.flatten())
print([(tokens[i], expl[i].item()) for i in range(len(tokens))])
vis_data_records = [visualization.VisualizationDataRecord(
                                expl,
                                output[0][classification],
                                classification,
                                1,
                                1,
                                1,       
                                tokens,
                                1)]
visualization.visualize_text(vis_data_records)

# Choosing class for visualization example

In [ ]:
# encode a sentence
text_batch = ["I hate that I love you."]
encoding = tokenizer(text_batch, return_tensors='pt')
input_ids = encoding['input_ids'].to(device)
attention_mask = encoding['attention_mask'].to(device)

# true class is positive - 1
true_class = 1

# generate an explanation for the input
target_class = 0
expl = explanations.generate_LRP(input_ids=input_ids, attention_mask=attention_mask, start_layer=11, index=target_class)[0]
# normalize scores
expl = (expl - expl.min()) / (expl.max() - expl.min())

# get the model classification
output = torch.nn.functional.softmax(model(input_ids=input_ids, attention_mask=attention_mask)[0], dim=-1)

# get class name
class_name = classifications[target_class]
# if the classification is negative, higher explanation scores are more negative
# flip for visualization
if class_name == "NEGATIVE":
  expl *= (-1)

tokens = tokenizer.convert_ids_to_tokens(input_ids.flatten())
print([(tokens[i], expl[i].item()) for i in range(len(tokens))])
vis_data_records = [visualization.VisualizationDataRecord(
                                expl,
                                output[0][classification],
                                classification,
                                true_class,
                                true_class,
                                1,       
                                tokens,
                                1)]
visualization.visualize_text(vis_data_records)

In [ ]:
# encode a sentence
text_batch = ["I hate that I love you."]
encoding = tokenizer(text_batch, return_tensors='pt')
input_ids = encoding['input_ids'].to(device)
attention_mask = encoding['attention_mask'].to(device)

# true class is positive - 1
true_class = 1

# generate an explanation for the input
target_class = 1
expl = explanations.generate_LRP(input_ids=input_ids, attention_mask=attention_mask, start_layer=11, index=target_class)[0]
# normalize scores
expl = (expl - expl.min()) / (expl.max() - expl.min())

# get the model classification
output = torch.nn.functional.softmax(model(input_ids=input_ids, attention_mask=attention_mask)[0], dim=-1)

# get class name
class_name = classifications[target_class]
# if the classification is negative, higher explanation scores are more negative
# flip for visualization
if class_name == "NEGATIVE":
  expl *= (-1)

tokens = tokenizer.convert_ids_to_tokens(input_ids.flatten())
print([(tokens[i], expl[i].item()) for i in range(len(tokens))])
vis_data_records = [visualization.VisualizationDataRecord(
                                expl,
                                output[0][classification],
                                classification,
                                true_class,
                                true_class,
                                1,       
                                tokens,
                                1)]
visualization.visualize_text(vis_data_records)